### IGNORE FOLLOWING CODE, JUST EXECTURE IT AND GO TO BLOCK BELOW

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timezone

class DownloadData(): 
    """
    Collection of methods that allows to donwload historical data trough Dukascopy API.
    """

    def __init__(self, ticker, start_date, end_date,timeframe):
        """
        Parameters
        ----------
        ticker : int
            Desired forex pair with format "XXX/YYY"
        start_date : str
            Start date with format "DD/MM/YYYY"
        end_date : str
            End date with format "DD/MM/YYYY"
        """
        timeframes = ['1day', '1hour', '10m','1min', '10sec', 'tick']
        if timeframe not in timeframes:
            raise SystemExit(f"Invalid timeframe, valid inputs are {timeframes}")
        else:
            self.timeframe = timeframe

        self.api_key = 'z3pstgimi8000000'
        self.ticker = ticker
        self.symbol = self.getTicker()
        
        # Date must be converted to UNIX with milliseconds
        self.start_date = datetime.timestamp(datetime.strptime(start_date, "%d-%m-%Y").replace(tzinfo=timezone.utc)) * 1000
        self.end_date = datetime.timestamp(datetime.strptime(end_date, "%d-%m-%Y").replace(tzinfo=timezone.utc)) * 1000

        if self.start_date > self.end_date:
            raise SystemExit(f"Start date can't be after end date.")

    def getTicker(self):
        url = 'https://freeserv.dukascopy.com/2.0/?path=api/instrumentList'
        params = {'key': self.api_key}
    
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = pd.DataFrame(response.json())
            filtered_data = data[data['name'] == self.ticker]
        else:
            print(f"Request failed with status code: {response.status_code}")

        if not filtered_data.empty:
            found_id = filtered_data['id'].values[0]
            del data, filtered_data, url, params, response
            return found_id
        else:
            raise SystemExit(f"No data found for name: {self.ticker}")

    def getData(self):

        url = 'https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices'

        download_progress = None
        final_data = pd.DataFrame()
        attempt = 0
        '''
        I had to create a while loop because the API let me download max. only 5000 rows of data at once. 
        So I create a loop that will do multiple requests, basically it donwload 5k rows of data at time 
        until all data relative to the desired datetime range has been donwloaded.
        '''
        while(True):
            '''
            If I want data from 01/01/2023 to today, then I'll be given only the most recent 5k rows of data. 
            Then I check if I already downloaded data into the dataframe, if yes less_recent_value assumes 
            the value of the first row Timestamp, so that this cycle will download the 5k rows of data that 
            precedes the ones already downlaoded. 
            '''
            params = {
                'key': self.api_key,
                'instrument': self.symbol,
                'timeFrame': self.timeframe,
                'count' : 5000, # Max rows of data downloadable, can't insert a value higher than this. 
                'start' : self.start_date,
                'end' : self.end_date if download_progress is None else download_progress
            }

            attempt = attempt+1
            print(f'\r \t Downloading... {self.ticker} '
                 f'{datetime.utcfromtimestamp(params["end"] / 1000)} '
                 f'Call n. {attempt}', end='', flush=True)

            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = pd.DataFrame(response.json())

                '''               
                There's a bug in the API: if you do rapidly many requests it can happen thtat
                you get data for an incorrect instrument. So this condition makes sure that
                the API provided the requested data, if not the code will continue to do
                requests until API deliever correct data. 
                '''

                if not data[data['id'] != self.symbol].empty:
                    continue 
                
                # Converting json in dataframe
                if params['timeFrame'] == 'tick':
                   data = pd.DataFrame(data['ticks'].apply(lambda x: {'Timestamp': x['timestamp'],'Bid': x['bid'],
                                                                       'Ask': x['ask']}).tolist())
                else:
                   data = pd.DataFrame(data['candles'].apply(lambda x: {'Timestamp': x['timestamp'],'Open': x['bid_open'], 
                                                                         'High': x['bid_high'], 'Low': x['bid_low'], 
                                                                         'Close': x['bid_close']}).tolist())

                # Appending data retrieved on this call to the main dataframe
                final_data = pd.concat([data, final_data], ignore_index=True)
            else:
                raise SystemExit(f"Request failed with status code: {response.status_code}")
            
            new_progress = final_data['Timestamp'].iloc[0]
            if not download_progress == new_progress: 
               download_progress = new_progress
            else:
               break
        
        # Clear print output
        print('\r' + ' ' * 80, end='\r', flush=True)

        # Check if there are duplicated rows
        if not final_data[final_data.duplicated()].empty:
            print("There are duplicated rows in the combined DataFrame.")

        # Convert timestamp to human-readable date and setting it as index
        final_data['Timestamp'] = pd.to_datetime(final_data['Timestamp'], unit='ms',utc=True)
        final_data.set_index('Timestamp',inplace=True)
        
        return final_data

# Without waiting for dukascopy, here's a larger dataset prepared for you and stored on google drive
!wget --header="Host: drive.usercontent.google.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7" --header="Cookie: CONSENT=PENDING+158; HSID=A_zy4wJzpjdNJy95e; APISID=EULozTaXaqtTjTE5/APB8wO0n3PBteVAXT; SSID=AymRbQgKxMShI-ZPF; SAPISID=lbgE4uLfnrjPzrvh/AM6ZThZaajTWXNSFG; __Secure-1PAPISID=lbgE4uLfnrjPzrvh/AM6ZThZaajTWXNSFG; __Secure-3PAPISID=lbgE4uLfnrjPzrvh/AM6ZThZaajTWXNSFG; S=billing-ui-v3=pa2ALjdE4gaNmik3nUcHnJMmWGdEFgMg:billing-ui-v3-efe=pa2ALjdE4gaNmik3nUcHnJMmWGdEFgMg:maestro=FC-n6MBeIgFQG-_9PyIVKhubSKCSZ3lGl9Up_WsDM10; SEARCH_SAMESITE=CgQI6pkB; SID=fAgR1a2tDJ8GIu2cKL80EbhN1tV2S4MNt0F-iuwnBM5ACoq-1EYZdfi3i5nJApwBz6FkTg.; __Secure-1PSID=fAgR1a2tDJ8GIu2cKL80EbhN1tV2S4MNt0F-iuwnBM5ACoq-CT9Twas0RnrlxzPwtE0t9w.; __Secure-3PSID=fAgR1a2tDJ8GIu2cKL80EbhN1tV2S4MNt0F-iuwnBM5ACoq-P0YVxpze0ifftTMEWr4VVQ.; 1P_JAR=2024-1-7-22; AEC=Ackid1QbKzi_84OGy0xlKK4zrdjIfQYsANWXggwTuyvlyIe0VQqVDij11A; __Secure-ENID=17.SE=DfRTVv6YAWPWQ_jQooO2Vi9ZjHEx5QKbbwGZ8nYHG6vErBZUBXXwnhlnCqcqx4HaHZWlcfP0d26zwm74MShpwnL28CwJdlv7t886eAq9h6QLRPHgHk-opMdBvrHn8qsExuEiLGMhFP_1J3DgIHsnfisz-b0Nc5Rl6sOUdgw3pYEbh_caxD6cVGWUOwT1skYtNbUgk74Q0c2h89dg9Uvux9X7hc8oRlkzvduZb43OOP9beM6H727MwIQ0qOcsDQVFBuzF7iB3sYLvl64bj9ohubi1UK0aUf36XzdD88Ink2usMNZ-3ZBKHhy3DczlvauG5kfx_nNqm9wfm86TjLuDJf_YgX-ZQ28m9ZBHia9s_qzZ-fBveBulZAX13ChBv6o5NCgrDg_p9hRPuoREkn5h_BC559Z6KKjtj8ePFapnruGwgqqxIA32UMNROnbA0ZOAorNHPDrUS7vR0QqEVxftXGyLq9eCY4U9CsCYq3Mc5K7Wn4guYtvsM_TY46hKR16eStEowow-hBRMCEfrchRrPoKm-YQH--eTnv_yUqwzREnBz18M-pnwMlkNfte-v4uilUSb7UdflY7W7fHaliFQc8yR2oyA5JWasseAcEtHQtbHT9bbFyXw; __Secure-1PSIDTS=sidts-CjEBPVxjSsWhW8slvaehfaPyT84U5jHoXrXg4nXoqXYN_RH80gW5DM446z8Qxu_ivKdqEAA; __Secure-3PSIDTS=sidts-CjEBPVxjSsWhW8slvaehfaPyT84U5jHoXrXg4nXoqXYN_RH80gW5DM446z8Qxu_ivKdqEAA; NID=511=ppm6fOxK20feNZ1AxeWFQfQJdSTaw-cW7XtANikwDrET8KzNm1C35wNfPDgREYgb9_72YzymvtX8XgRf_IAdiBJgjJ6zdUIUeNllxlowDLZWDPB8S2llawqDe4VQKFrP2tU-WSJjH1ffldMHXcou2NsVkqPZt31J9NZrDKzSR4GIAoMClw1wV6llpwAjI-OQ6rsnAYUAJylGSerl-vMqFKvRDEKap2zEBbJaWSdN61wfqbIFLnAJaKMYPjW3OQT9mOtJMyTuPwKY0TzCspJosNom84kyipU8DzZ9J1_eywidOD8x7Edrk9eLz2R210V4guIE55d-9F7fVSqbLQnI8TX2894vfKitqA8dBfs8cI4HPSoOXg9_ud6YSmsx5_VzOIL4IeAbhAyPJSWp6D5gk4GqZRRK02RJaWgYWIxtOZvsbny4MIU9-E5-UUV6p1Dw8zaipNMDNYxK53lwrQKUMBP374b9GTqZHpAe-Kw4JfjAawYTZp0n9Mwp5vak5HIJ8mdyubJfFgLZhTSnzPZ_h01ljVl20ASyebu9C4qlvESvW7yZ4f6o1712V9VtrK45tX8SMbtLSpItpgtqs7zOjCGRqdZsNK38tgagcyQfuB1y-F3N2w; SIDCC=ABTWhQEYU6MNss_qdXlrLu0DepvwpqJJmQLX8tcUM84Cs4T6JyjIKOnBLvPNmN8lp6pFa8GP6YM; __Secure-1PSIDCC=ABTWhQGdsU_jEcZ71iwvniNQp28Tjhnhn1TdDAer2DvAaVm7mVXXrvDIyagBzSehW8jrRUIYNeE; __Secure-3PSIDCC=ABTWhQGHmM3HEDh47a-vUMvACU2bgeSYWyD53fIXZhKG0-zanFf-pyf034SVw66D5-dOXuGWwKJ2" --header="Connection: keep-alive" "https://drive.usercontent.google.com/download?id=1CzEZwV0TsBUt8wt_IpC-CX133RIt0Y8y&export=download&authuser=0&confirm=t&uuid=200d723e-bc88-4373-8e89-283936018f0b&at=APZUnTU2s3YoPVGN1qbjeCyste4V:1704722361106" -c -O '01-12-2023_05-01-2024 tick.csv'

# Classify price movements based on candlesticks statistics

This reseach comes from [this](https://www.forexfactory.com/thread/post/14707863#post14707863) post on ForexFactory. 

### Step 1: gather data and create a rolling TF

In [1]:
# This allow jupiter to upload in real time externally modified code
%load_ext autoreload
%autoreload 2 


#import sys
#sys.path.append("..")
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
#from x_CLASSES.download_data import DownloadData


In [2]:
start_date = "01-12-2023"
end_date = "05-01-2024"
timeframe = 'tick'
price_frame = 1000

# You can upload your own data, just make sure to update start_date and end_date, and also that
# the csv filename format match the template
csv_file_path = f"{start_date}_{end_date} {timeframe}.csv"

df = pd.DataFrame()


if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
else:
    df = DownloadData('GBP/USD', start_date,end_date,timeframe).getData()
    
    data_folder_path = f"{os.getcwd()}/x_DATA"
    if not os.path.exists(data_folder_path):
        os.makedirs(data_folder_path)

    df.to_csv(csv_file_path)

df

,Timestamp,Bid,Ask
0,2023-12-01 00:00:02.033000+00:00,1.26305,1.26316
1,2023-12-01 00:00:05.974000+00:00,1.26306,1.26316
2,2023-12-01 00:00:07.587000+00:00,1.26305,1.26316
3,2023-12-01 00:00:07.689000+00:00,1.26306,1.26316
4,2023-12-01 00:00:07.792000+00:00,1.26304,1.26317
...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26820
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26821
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26820
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26819


In [3]:
if "Ask" in df.columns:
    df = df.drop("Ask", axis=1)

df["Close"] = df["Bid"].copy()
df["Open"] = df["Bid"].shift(price_frame)  # Shift the "Bid" values 1000 rows back
df["High"] = df["Bid"].rolling(window=price_frame).max()  # Calculate the rolling max over the last 1000 rows
df["Low"] = df["Bid"].rolling(window=price_frame).min()  # Calculate the rolling min over the last 1000 rows

df

,Timestamp,Bid,Close,Open,High,Low
0,2023-12-01 00:00:02.033000+00:00,1.26305,1.26305,NaN,NaN,NaN
1,2023-12-01 00:00:05.974000+00:00,1.26306,1.26306,NaN,NaN,NaN
2,2023-12-01 00:00:07.587000+00:00,1.26305,1.26305,NaN,NaN,NaN
3,2023-12-01 00:00:07.689000+00:00,1.26306,1.26306,NaN,NaN,NaN
4,2023-12-01 00:00:07.792000+00:00,1.26304,1.26304,NaN,NaN,NaN
...,...,...,...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26810,1.26834,1.26856,1.26795
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26808,1.26833,1.26856,1.26795
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26808,1.26835,1.26856,1.26795
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26805,1.26840,1.26856,1.26795


#### Time of high and time of low
I'm adding a column that returns the high and low times.

In [8]:
# Calculate the index of the maximum value in the rolling window for "High time"
df['High time'] =  price_frame - ( df.index.values - df['Bid'].rolling(window=price_frame).agg(lambda x: x.index.values[np.argmax(x.values)]) ) 
df["Low time"] = price_frame  - ( df.index.values - df['Bid'].rolling(window=price_frame).agg(lambda x: x.index.values[np.argmin(x.values)]) )
df['High first'] = df['High time'] < df["Low time"]

df = df.dropna() #Drop initial NaN values

df

,Timestamp,Bid,Close,Open,High,Low,High time,Low time,High first,Open - Close,Upper Wick,Lower Wick,Bias,Strength
1999,2023-12-01 00:43:00.300000+00:00,1.26398,1.26398,1.26376,1.26417,1.26371,477.00000,13.00000,False,0.00022,0.00019,0.00005,Bullish,Weak
2000,2023-12-01 00:43:00.602000+00:00,1.26398,1.26398,1.26378,1.26417,1.26371,476.00000,12.00000,False,0.00020,0.00019,0.00007,Bullish,Weak
2001,2023-12-01 00:43:01.108000+00:00,1.26398,1.26398,1.26377,1.26417,1.26371,475.00000,11.00000,False,0.00021,0.00019,0.00006,Bullish,Weak
2002,2023-12-01 00:43:01.312000+00:00,1.26398,1.26398,1.26378,1.26417,1.26371,474.00000,10.00000,False,0.00020,0.00019,0.00007,Bullish,Weak
2003,2023-12-01 00:43:02.070000+00:00,1.26399,1.26399,1.26378,1.26417,1.26371,473.00000,9.00000,False,0.00021,0.00018,0.00007,Bullish,Weak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26810,1.26834,1.26856,1.26795,255.00000,119.00000,False,0.00024,0.00022,0.00015,Bearish,Weak
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26808,1.26833,1.26856,1.26795,254.00000,118.00000,False,0.00025,0.00023,0.00013,Bearish,Weak
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26808,1.26835,1.26856,1.26795,253.00000,117.00000,False,0.00027,0.00021,0.00013,Bearish,Weak
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26805,1.26840,1.26856,1.26795,252.00000,116.00000,False,0.00035,0.00016,0.00010,Bearish,Weak


#### How to interpret 'High time' and 'Low time' columns
e.g. 
- High time = 746 and Low time = 200
- That means: the high was made on tick number 746 of the current 1000 ticks candlestick

Here's a draw so that you can fully understand
![Drawing](https://i.imgur.com/ORcChmJ.png)

### Step 2: Directional bias

In [5]:
df['Open - Close'] = (df['Open'] - df['Close']).abs()
df['Upper Wick'] = (df['High'] - df[['Open', 'Close']].max(axis=1))
df['Lower Wick'] = (df[['Open', 'Close']].min(axis=1) - df['Low'])

df['Bias'] = np.where(df['Open'] > df['Close'], 'Bearish', np.where(df['Close'] > df['Open'], 'Bullish', 'Doji'))

df


,Timestamp,Bid,Close,Open,High,Low,High time,Low time,High first,Open - Close,Upper Wick,Lower Wick,Bias
1000,2023-12-01 00:21:58.140000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298,820.0,14.0,True,0.00073,0.00019,0.00007,Bullish
1001,2023-12-01 00:22:00.557000+00:00,1.26377,1.26377,1.26306,1.26397,1.26298,819.0,13.0,True,0.00071,0.00020,0.00008,Bullish
1002,2023-12-01 00:22:00.912000+00:00,1.26378,1.26378,1.26305,1.26397,1.26298,818.0,12.0,True,0.00073,0.00019,0.00007,Bullish
1003,2023-12-01 00:22:01.166000+00:00,1.26378,1.26378,1.26306,1.26397,1.26298,817.0,11.0,True,0.00072,0.00019,0.00008,Bullish
1004,2023-12-01 00:22:01.317000+00:00,1.26378,1.26378,1.26304,1.26397,1.26298,816.0,10.0,True,0.00074,0.00019,0.00006,Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26810,1.26834,1.26856,1.26795,255.0,119.0,True,0.00024,0.00022,0.00015,Bearish
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26808,1.26833,1.26856,1.26795,254.0,118.0,True,0.00025,0.00023,0.00013,Bearish
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26808,1.26835,1.26856,1.26795,253.0,117.0,True,0.00027,0.00021,0.00013,Bearish
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26805,1.26840,1.26856,1.26795,252.0,116.0,True,0.00035,0.00016,0.00010,Bearish


### Step 3: Strength of directional bias
On this version boundaries will be calculated trough average directional OC and average directional wick height. It can be:
- weak: $0<$ OC $<=$ Average OC
- medium ("normal"): Average OC $<$ OC $<=$ Average wick height
- strong: Average wick height $<$ OC

Here's a better rappresentation of the possible categories:
![](https://i.imgur.com/EnRJAK1.png)

In [10]:
# Filter rows where Bias is "Bullish"
bullish_rows = df[df['Bias'] == 'Bullish']
bearish_rows = df[df['Bias'] == 'Bearish']

# average directional OC
average_bullish_oc = bullish_rows['Open - Close'].mean()
average_bearish_oc = bearish_rows['Open - Close'].mean()
avg_oc = ( average_bullish_oc + average_bearish_oc ) / 2

# average bullish and bearish candle Wick (relative to close)
average_bullish_upper = bullish_rows['Upper Wick'].mean()
average_bearish_lower = bearish_rows['Lower Wick'].mean()
avg_wk = avg_oc + ( average_bullish_upper + average_bearish_lower ) / 2

# assign strenght bias
df['Strength'] = np.where(df['Open - Close'] > avg_wk, 'Strong',
                          np.where((df['Open - Close'] > avg_oc) & (df['Open - Close'] <= avg_wk), 'Medium', 'Weak'))

del avg_oc, avg_wk
df


,Timestamp,Bid,Close,Open,High,Low,High time,Low time,High first,Open - Close,Upper Wick,Lower Wick,Bias,Strength
1999,2023-12-01 00:43:00.300000+00:00,1.26398,1.26398,1.26376,1.26417,1.26371,477.00000,13.00000,False,0.00022,0.00019,0.00005,Bullish,Weak
2000,2023-12-01 00:43:00.602000+00:00,1.26398,1.26398,1.26378,1.26417,1.26371,476.00000,12.00000,False,0.00020,0.00019,0.00007,Bullish,Weak
2001,2023-12-01 00:43:01.108000+00:00,1.26398,1.26398,1.26377,1.26417,1.26371,475.00000,11.00000,False,0.00021,0.00019,0.00006,Bullish,Weak
2002,2023-12-01 00:43:01.312000+00:00,1.26398,1.26398,1.26378,1.26417,1.26371,474.00000,10.00000,False,0.00020,0.00019,0.00007,Bullish,Weak
2003,2023-12-01 00:43:02.070000+00:00,1.26399,1.26399,1.26378,1.26417,1.26371,473.00000,9.00000,False,0.00021,0.00018,0.00007,Bullish,Weak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383418,2024-01-04 23:59:36.774000+00:00,1.26810,1.26810,1.26834,1.26856,1.26795,255.00000,119.00000,False,0.00024,0.00022,0.00015,Bearish,Weak
2383419,2024-01-04 23:59:41.153000+00:00,1.26808,1.26808,1.26833,1.26856,1.26795,254.00000,118.00000,False,0.00025,0.00023,0.00013,Bearish,Weak
2383420,2024-01-04 23:59:41.256000+00:00,1.26808,1.26808,1.26835,1.26856,1.26795,253.00000,117.00000,False,0.00027,0.00021,0.00013,Bearish,Weak
2383421,2024-01-04 23:59:55.951000+00:00,1.26805,1.26805,1.26840,1.26856,1.26795,252.00000,116.00000,False,0.00035,0.00016,0.00010,Bearish,Weak


### Step 4: print results

In [11]:
from tabulate import tabulate

# Possible values in each column
unique_biases = ['Bullish','Doji','Bearish']
unique_strengthes = ['Strong','Medium','Weak']
unique_high_first_values = df['High first'].unique()

# Create a list to store the results
result_data = []

pd.options.display.float_format = '{:.5f}'.format

# Iterate over combinations and calculate mean
for bias in unique_biases:
    for strength in unique_strengthes:
        for high_first in unique_high_first_values:
            subset = df[(df['Bias'] == bias) & (df['Strength'] == strength) & (df['High first'] == high_first)]
            sample = ( subset.size / df.size ) *100

            if sample >1: # I do not consider configurations that happenes less than 1% of times
                
                # Average OC
                mean_oc = subset['Open - Close'].mean()
                mean_oc = mean_oc
                # Average Higher and Lower wick
                mean_uwk = subset['Upper Wick'].mean()
                mean_lwk = subset['Lower Wick'].mean()
                # Average time of High and time of Low
                mean_thigh = int(round(subset['High time'].mean(),0))
                mean_tlow = int(round(subset['Low time'].mean(),0))
                time_ = f'{mean_thigh} - {mean_tlow}'

                result_data.append({'Sampple':f'{round(sample,2)}%','Bias': bias, 'Strength': strength, 'High First': high_first, 'Avg. OC': mean_oc,
                                    'Avg. Upper Wick': mean_uwk, 'Avg. Lower Wick':mean_lwk,'Avg. Time High - Low': time_})

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(result_data)

# Print the formatted table
print(tabulate(result_df, headers='keys', tablefmt='simple_outline', showindex=False, floatfmt=".5f", numalign="center", stralign="center"))



┌───────────┬─────────┬────────────┬──────────────┬───────────┬───────────────────┬───────────────────┬────────────────────────┐
│  Sampple  │  Bias   │  Strength  │  High First  │  Avg. OC  │  Avg. Upper Wick  │  Avg. Lower Wick  │  Avg. Time High - Low  │
├───────────┼─────────┼────────────┼──────────────┼───────────┼───────────────────┼───────────────────┼────────────────────────┤
│  11.16%   │ Bullish │   Strong   │    False     │  0.00105  │      0.00016      │      0.00016      │       879 - 118        │
│   8.37%   │ Bullish │   Medium   │    False     │  0.00056  │      0.00018      │      0.00018      │       816 - 160        │
│  24.35%   │ Bullish │    Weak    │    False     │  0.00023  │      0.00024      │      0.00023      │       733 - 226        │
│   5.59%   │ Bullish │    Weak    │     True     │  0.00013  │      0.00027      │      0.00027      │       296 - 678        │
│  11.41%   │ Bearish │   Strong   │     True     │  0.00104  │      0.00015      │      0.00016 